### Import Modules

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Weaviate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### Load the Document

In [2]:
with open("./docs/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

Created a chunk of size 304, which is longer than the specified 300
Created a chunk of size 332, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300


### Fill the Vectore Store

In [3]:
WEAVIATE_URL = "http://127.0.0.1:8080"
embeddings = OllamaEmbeddings(model="llama2:13b")
docsearch = Weaviate.from_texts(
    texts,
    embeddings,
    weaviate_url=WEAVIATE_URL,
    by_text=False,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))],
)

retriever = docsearch.as_retriever()

### Create a Prompt

In [4]:
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


### Set up the LLM to use

In [5]:
llm = Ollama(model="llama2:13b")

### Run the chain

In [7]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What did the president say about Ketanji Brown Jackson")

'Based on the provided context, here is the answer to your question:\n\nThe president nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court.'